# Parameter Identification Technique (PIT) on a linear Nomoto model

# Purpose
Show the general workflow by applying PIT regression on a very simple model

# Methodology
* Load time series from a ZigZag test
* Determine yaw rate and acceleration (compare with accelerometers from test).
* Find the best parameter values in the Nomoto model using OLS linear regression.

# Setup

In [8]:
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
import pyro

import sklearn
import pykalman

## Local packages:
from src.data import mdl




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load test

In [9]:
df_runs = mdl.runs()

In [10]:
df = mdl.load_run(id=22775)

In [11]:
df.head()

,Arr/Ind/Arr,Arr/Ind/Fri,Fan/Aft/Angle,Fan/Aft/AngleOrder,Fan/Aft/CalcAngle,Fan/Aft/Fx,Fan/Aft/FxOrder,Fan/Aft/Rpm,Fan/Aft/RpmOrder,Fan/ForceFactor,Fan/Fore/Angle,Fan/Fore/AngleOrder,Fan/Fore/CalcAngle,Fan/Fore/Fx,Fan/Fore/FxOrder,Fan/Fore/Rpm,Fan/Fore/RpmOrder,Hull/Acc/X1,Hull/Acc/Y1,Hull/Acc/Y2,Hull/Acc/Z1,Hull/Acc/Z2,Hull/Acc/Z3,Prop/PS/Rpm,Prop/PS/Thrust,Prop/PS/Torque,Prop/SB/Rpm,Prop/SB/Thrust,Prop/SB/Torque,Rudder/Angle,Rudder/MaxAngle,Rudder/Rate,Wind/Course,Wind/CourseTreshold,Wind/Driftvinkel,Wind/Enable,Wind/GWA,Wind/Heading,lab/WaveHeight,roll,pitch,psi,x0,y0,z0
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0 days 00:00:26.669996,0.0,1.0,0.363455,0.363314,0.363314,6.350474,9.997757,39.535089,41.287582,0.62,0.681240,0.680888,0.680888,5.975769,9.997757,34.449519,42.060705,0.305876,0.200020,-0.192392,0.003395,0.214486,0.123971,5.525132,-0.059149,0.020355,5.497526,-0.010540,-0.018574,0.157073,0.645772,14.89,0.001976,0.3,-2.3,1.0,-1.570796,6.274633,0.001672,0.005155,0.000360,6.234491,7.305435,0.005102,0.002555
0 days 00:00:26.679998,0.0,1.0,0.363359,0.363298,0.363298,6.619624,9.997551,39.207038,41.287159,0.62,0.680858,0.680857,0.680857,6.150300,9.997551,26.209653,42.060281,-0.085527,-0.159691,-0.234983,0.205034,0.663210,-0.421227,5.520644,-0.074466,0.016496,5.510466,-0.056368,-0.021457,0.157073,0.645772,14.89,0.002014,0.3,-2.3,1.0,-1.570796,6.274459,0.001547,0.005173,-0.000614,6.234316,7.314149,0.005140,0.002317
0 days 00:00:26.690006,0.0,1.0,0.363420,0.363283,0.363283,6.626957,9.997345,40.499945,41.286739,0.62,0.681226,0.680827,0.680827,6.242503,9.997345,29.335785,42.059862,0.053753,-0.121012,0.143173,-0.066824,0.115844,-0.607471,5.482439,-0.088359,0.012905,5.509347,-0.094498,-0.022990,0.157073,0.645772,14.89,0.002051,0.3,-2.3,1.0,-1.570796,6.274284,0.001497,0.005144,0.000368,6.234142,7.325029,0.005271,0.002558
0 days 00:00:26.699996,0.0,1.0,0.363311,0.363267,0.363267,6.354853,9.997144,39.091256,41.286329,0.62,0.680885,0.680797,0.680797,6.274332,9.997144,41.570152,42.059451,0.371647,0.257393,0.326443,-0.603454,-0.441839,0.971412,5.495299,-0.103247,0.010246,5.502427,-0.137763,-0.023119,0.157073,0.645772,14.89,0.002087,0.3,-2.3,1.0,-1.570796,6.274022,0.001522,0.005171,-0.000530,6.233880,7.343141,0.005337,0.002425
0 days 00:00:26.710002,0.0,1.0,0.363380,0.363252,0.363252,5.976993,9.996948,38.878988,41.285927,0.62,0.681035,0.680768,0.680768,6.179950,9.996948,26.035979,42.059049,-0.101002,-0.477500,0.048957,0.215986,-0.194267,0.049216,5.496015,-0.126295,0.008693,5.506842,-0.171454,-0.022048,0.157079,0.645772,14.89,0.002123,0.3,-2.3,1.0,-1.570796,6.274022,0.001547,0.005163,0.000324,6.233880,7.343305,0.005332,0.002463


In [22]:
units = mdl.load_units()